# Intuition for Text Generation

This notebook shows an end to end process for the objective of generating text on a character level. It first reads a text file as defined in `filename` and performs the following:

* Text Cleanup by lowercasing everything
* Text Cleanup by removing unwanted symbols
* Creating a mapping for character indices and indices to characters
* Defining x and y vectors for classification
* Pass it to a multi-layer perceptron
* Generate some text

## Required Libraries

* `nltk`
* `scikit-learn`
* `torch`

### Import the Necessary Libraries

In [1]:
import re
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ralampay/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Read a Text File and Get Raw Text

You may change `filename` to the location of your own file.

In [2]:
filename = "story.txt"
f = open(filename, 'r')

raw_text = f.read()

### Examine First 1000 Characters

In [3]:
raw_text[0:1000]

'Once upon a time, there was a boy named Mike\nHe was raised in the hood, life was never quite\nGrowing up, he faced many struggles and strife\nBut through it all, he had one love in his life\n\nRap music was his escape from reality\nThe beats and lyrics helped him to see\nA world beyond his troubled neighborhood\nHe knew that with hard work, he could change his mood\n\nHe started writing rhymes and practicing his flow\nIn the mirror, he would rap and watch himself grow\nHe knew that if he could just make it to the top\nHe could change his life, and make a better stop\n\nOne day, he had the chance to perform on stage\nIn front of a crowd, he killed it, he killed it with rage\nHe had finally made it, he had achieved his dream\nAnd now he is a successful rapper, or so it seems\n\nHe never forgot where he came from, and he never will\nHe always remember the struggles that he had to fulfill\nHe is now a role model to kids in the hood\nShowing them that with hard work, they too could\n\nRis

### Cleanup Text

* Lowercase all characters
* Remove special symbols

Note that this retains things like spaces, commas, periods and other punctuations.

In [4]:
processed_text = raw_text.lower()
processed_text = re.sub(r'[^\x00-\x7f]', r'', processed_text)

processed_text

'once upon a time, there was a boy named mike\nhe was raised in the hood, life was never quite\ngrowing up, he faced many struggles and strife\nbut through it all, he had one love in his life\n\nrap music was his escape from reality\nthe beats and lyrics helped him to see\na world beyond his troubled neighborhood\nhe knew that with hard work, he could change his mood\n\nhe started writing rhymes and practicing his flow\nin the mirror, he would rap and watch himself grow\nhe knew that if he could just make it to the top\nhe could change his life, and make a better stop\n\none day, he had the chance to perform on stage\nin front of a crowd, he killed it, he killed it with rage\nhe had finally made it, he had achieved his dream\nand now he is a successful rapper, or so it seems\n\nhe never forgot where he came from, and he never will\nhe always remember the struggles that he had to fulfill\nhe is now a role model to kids in the hood\nshowing them that with hard work, they too could\n\nris

### Create Index Mappings

* `character_indices`: Return the index of a given character. Example: `character_indices['a']`
* `indices_characters`: Returns the character given an index. Example: `indices_characters[0]`

In this case, `characters` serves as your **vocabulary**.

In [5]:
print("Corpus Length: {}".format(len(processed_text)))

characters = sorted(list(set(processed_text)))
print("Total Characters: {}".format(len(characters)))

character_indices = dict((c, i) for i,c in enumerate(characters))
indices_characters = dict((i, c) for i, c in enumerate(characters))

Corpus Length: 1073
Total Characters: 27


In [6]:
character_indices

{'\n': 0,
 ' ': 1,
 ',': 2,
 'a': 3,
 'b': 4,
 'c': 5,
 'd': 6,
 'e': 7,
 'f': 8,
 'g': 9,
 'h': 10,
 'i': 11,
 'j': 12,
 'k': 13,
 'l': 14,
 'm': 15,
 'n': 16,
 'o': 17,
 'p': 18,
 'q': 19,
 'r': 20,
 's': 21,
 't': 22,
 'u': 23,
 'v': 24,
 'w': 25,
 'y': 26}

In [7]:
indices_characters

{0: '\n',
 1: ' ',
 2: ',',
 3: 'a',
 4: 'b',
 5: 'c',
 6: 'd',
 7: 'e',
 8: 'f',
 9: 'g',
 10: 'h',
 11: 'i',
 12: 'j',
 13: 'k',
 14: 'l',
 15: 'm',
 16: 'n',
 17: 'o',
 18: 'p',
 19: 'q',
 20: 'r',
 21: 's',
 22: 't',
 23: 'u',
 24: 'v',
 25: 'w',
 26: 'y'}

### Convert to a Set of Symbols of Fixed Length

* `maxlen`: Dimensionality of each data point
* `step`: Granularity of skips. The lower the number, the noisier. The higher the number, the more erratic.

Take note we also capture the predicted character for the given sentence. This will allow us to setup the data in such a way that a given set of sequences predicts the next character. In machine learning, we will denote this as our `y` value or ground truth. Each `y` however is represented as a one hot encoding where a position will receive a value of `1` depending on the character position in `character_indices`

In [8]:
maxlen = 40
step = 5

sentences = []
next_characters = []

for i in range(0, len(processed_text) - maxlen, step):
    sentences.append(processed_text[i: i+maxlen])
    next_characters.append(processed_text[i + maxlen])

In [9]:
sentences

['once upon a time, there was a boy named ',
 'upon a time, there was a boy named mike\n',
 'a time, there was a boy named mike\nhe wa',
 'e, there was a boy named mike\nhe was rai',
 'ere was a boy named mike\nhe was raised i',
 'as a boy named mike\nhe was raised in the',
 'boy named mike\nhe was raised in the hood',
 'amed mike\nhe was raised in the hood, lif',
 'mike\nhe was raised in the hood, life was',
 'he was raised in the hood, life was neve',
 's raised in the hood, life was never qui',
 'sed in the hood, life was never quite\ngr',
 'n the hood, life was never quite\ngrowing',
 ' hood, life was never quite\ngrowing up, ',
 ', life was never quite\ngrowing up, he fa',
 'e was never quite\ngrowing up, he faced m',
 ' never quite\ngrowing up, he faced many s',
 'r quite\ngrowing up, he faced many strugg',
 'te\ngrowing up, he faced many struggles a',
 'owing up, he faced many struggles and st',
 ' up, he faced many struggles and strife\n',
 'he faced many struggles and strife\n

In [10]:
next_characters

['m',
 'h',
 's',
 's',
 'n',
 ' ',
 ',',
 'e',
 ' ',
 'r',
 't',
 'o',
 ' ',
 'h',
 'c',
 'a',
 't',
 'l',
 'n',
 'r',
 'b',
 'h',
 'h',
 'a',
 'h',
 'd',
 ' ',
 ' ',
 'i',
 'f',
 'a',
 's',
 'a',
 's',
 'a',
 'r',
 'e',
 'y',
 ' ',
 's',
 ' ',
 'c',
 'l',
 'h',
 'o',
 '\n',
 'r',
 'e',
 ' ',
 't',
 'l',
 'e',
 'o',
 'd',
 'k',
 't',
 'w',
 'h',
 'w',
 ' ',
 'o',
 'c',
 'e',
 ' ',
 '\n',
 's',
 'e',
 'i',
 ' ',
 'e',
 'd',
 'c',
 'n',
 's',
 'w',
 't',
 'i',
 ',',
 'w',
 ' ',
 'a',
 'a',
 'h',
 'l',
 'o',
 ' ',
 ' ',
 ' ',
 'e',
 'l',
 's',
 'k',
 ' ',
 'h',
 'p',
 'c',
 ' ',
 'g',
 's',
 'e',
 'd',
 'e',
 'e',
 ' ',
 '\n',
 ' ',
 ' ',
 'a',
 'e',
 'n',
 'o',
 'f',
 'o',
 'a',
 'n',
 'n',
 ' ',
 'o',
 'h',
 'l',
 'i',
 'e',
 'l',
 't',
 'h',
 'e',
 'h',
 'i',
 'y',
 'e',
 ' ',
 'a',
 'h',
 'd',
 ' ',
 'm',
 ' ',
 'h',
 ' ',
 'c',
 'f',
 'a',
 ',',
 's',
 ' ',
 's',
 ' ',
 'r',
 'g',
 'h',
 'h',
 'm',
 'o',
 'n',
 ' ',
 'r',
 'l',
 'a',
 's',
 'e',
 ' ',
 's',
 'g',
 't',
 'h',
 'd',
 

### Vectorization

This step simply converts the `sentences` and `next_characters` to its `x` and `y` components respectively. Since we're using pytorch, we convert it to a tensor. Succeeding cells check the shape of `x` and `y`.

In [11]:
print("Vectorization")
device = 'cuda'
x = np.zeros((len(sentences), maxlen, len(characters)), dtype=np.float64)
y = np.zeros((len(sentences), len(characters)), dtype=np.float64)

for i, sentence in enumerate(sentences):
    for t, character, in enumerate(sentence):
        x[i, t, character_indices[character]] = 1
    y[i, character_indices[next_characters[i]]] = 1
    
x = torch.tensor(x).float().to(device)
y = torch.tensor(y).float().to(device)

Vectorization


In [12]:
x.shape

torch.Size([207, 40, 27])

In [13]:
x = torch.flatten(x, start_dim=1)

x.shape

torch.Size([207, 1080])

In [14]:
y.shape

torch.Size([207, 27])

### Utility Function for Generating Samples

In [15]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

### Callback Function

This generates sample text from a given seed and ran for every epoch of the model.

In [16]:
def callback(model):
    start = 0
    stop = len(processed_text) - maxlen - 1

    #print("Start: {}".format(start))
    #print("Stop: {}".format(stop))

    start_index = random.randint(start, stop)

    #print("Start Index: {}".format(start_index))

    sentence = processed_text[start_index: start_index + maxlen]

    #print("Sentence: {}".format(sentence))
    #print("Sentence Length: {}".format(len(sentence)))

    generated = ''

    for i in range(400):
        x_predictions = np.zeros((1, maxlen, len(characters)))

        for t, char in enumerate(sentence):
            x_predictions[0, t, character_indices[char]] = 1

            # print(x_predictions)
        x_predictions = torch.tensor(x_predictions).float().to(device)
        x = torch.flatten(x_predictions, start_dim=1)

        preds = model.forward(x)[0].detach().cpu().numpy()

        next_index = sample(preds)
        #print("next_index: {}".format(next_index))
        next_char = indices_characters[next_index]
        #print("next_char: {}".format(next_char))

        generated += next_char
        sentence = sentence[1:] + next_char

    return sentence

### MultiLayerPerceptron Model

This will be our current language model. Although originally used for classification, we can also treat is a regression model since our ground truth represents the next predicted character in a sequence. Take note that this is a rather simplistic model without any properties to remember previous input. This forces the model to treat each input as an independent observation without considering sequential behavior. TLDR, it won't generate good results.

In [17]:
class MultiLayerPerceptron(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()

        self.hidden = nn.Linear(input_dim, 500)
        self.output = nn.Linear(500, output_dim)
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # f(x) = a(f(x))
        x = self.relu(self.hidden(x))
        y = self.sigmoid(self.output(x))

        return y

In [18]:
model = MultiLayerPerceptron(x.shape[1], y.shape[1]).to(device)

model

MultiLayerPerceptron(
  (hidden): Linear(in_features=1080, out_features=500, bias=True)
  (output): Linear(in_features=500, out_features=27, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)

### Training Function

In [19]:
optimizer = optim.Adam(model.parameters(), lr=0.00001)
criterion = nn.CrossEntropyLoss()

def train_fn(model, optimizer, loss_fn, device):
    ave_loss = 0
    count = 0
    
    for i, data in enumerate(x):
        data = x[i]
        targets = y[i]
        
        # Forward
        predictions = model.forward(data)
        
        predictions = F.softmax(predictions, dim=-1)
        
        loss = loss_fn(predictions, targets)
        
        # Backward
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()

        count += 1
        ave_loss += loss.item()
    
    ave_loss = ave_loss / count

    return ave_loss

epochs = 1000

average_losses = []

for epoch in range(epochs):
    print("Epoch: {}".format(epoch))
    ave_loss = train_fn(model, optimizer, criterion, device)
    
    average_losses.append(ave_loss)
        
    print("Ave Loss: {}".format(ave_loss))
    
    generated_sentence = callback(model)
    
    print("Generated sentence:")
    print(generated_sentence)
    print("Length: {}".format(len(generated_sentence)))

Epoch: 0
Ave Loss: 3.295721020675512
Generated sentence:
ks m nqndn
o,om cd
wy,bt,wshtkjq
fpnqopi
Length: 40
Epoch: 1
Ave Loss: 3.295455629698896
Generated sentence:
,aeiotl bd,fvwclug,lf,klfq
v bvewfhi
,qi
Length: 40
Epoch: 2
Ave Loss: 3.295182005794728
Generated sentence:
mjtjsmhruteuoio,hpms
 hosetia pshu lc
vs
Length: 40
Epoch: 3
Ave Loss: 3.294897715250651
Generated sentence:
idvdyih
uro,s
euvciydqcyufhyi khsg
mavsk
Length: 40
Epoch: 4
Ave Loss: 3.2946004729340044
Generated sentence:
yjlh , f
huqvshocswdyvlg,q at oph
rhhcal
Length: 40
Epoch: 5
Ave Loss: 3.294288440602989
Generated sentence:
kbcquiijerkeoh,hb,fd,ruq,owpjtesrmtavbqr
Length: 40
Epoch: 6
Ave Loss: 3.293959275535915
Generated sentence:
eeryrnsccutamujej nfvu,dyghoaohbdhprbiof
Length: 40
Epoch: 7
Ave Loss: 3.293609266695769
Generated sentence:
arjwyganalkuttonoclir cbaveoorvqbhsuywce
Length: 40
Epoch: 8
Ave Loss: 3.293236609242389
Generated sentence:
wepd r,h wg,lhsopbp vioof wt
bmcayecesky
Length: 40
Epoch: 9
Ave Loss:

Ave Loss: 3.2647440940285652
Generated sentence:
o snel ioa  eidoneh  rs dhcnlnllnstki ad
Length: 40
Epoch: 76
Ave Loss: 3.2643312912632303
Generated sentence:
fohanrrth lma bleeltwdshr tonn osslt ese
Length: 40
Epoch: 77
Ave Loss: 3.263916734336079
Generated sentence:
o toa mhehtahe cschehanr   roellas hin h
Length: 40
Epoch: 78
Ave Loss: 3.263502260337129
Generated sentence:
rlh ea  stssho aisi elrwhhliertnseah ton
Length: 40
Epoch: 79
Ave Loss: 3.263088229773701
Generated sentence:
aesetwa
eti o l,lessaceati  noyh trlldlc
Length: 40
Epoch: 80
Ave Loss: 3.26267373273914
Generated sentence:
lisaoh  a sltdhhh n
s si tte hhoot l
shi
Length: 40
Epoch: 81
Ave Loss: 3.2622602700035355
Generated sentence:
ai dsieaihotllsneeiradhwiat  eae    i i 
Length: 40
Epoch: 82
Ave Loss: 3.2618470952130747
Generated sentence:
see eeelief oan hhe ssnnheis nhtsonhgi
 
Length: 40
Epoch: 83
Ave Loss: 3.2614357667268763
Generated sentence:
swhm oyee lsohheha ioa wrl rers  
f sesa
Length: 40
Epoch: 84
Ave Lo

Ave Loss: 3.2425212422430802
Generated sentence:
f he   f aah 
ee ami    ca ein  eane  e 
Length: 40
Epoch: 150
Ave Loss: 3.24238742031337
Generated sentence:
ie hasr os tea ha h is  hhoiila hthnt  h
Length: 40
Epoch: 151
Ave Loss: 3.242257274867256
Generated sentence:
 a ha eoc ae h o     s  o  e ra a se  i 
Length: 40
Epoch: 152
Ave Loss: 3.2421304914686413
Generated sentence:
ha   dnse a e      e hi    h  e eie  ae 
Length: 40
Epoch: 153
Ave Loss: 3.24200757690098
Generated sentence:
 h eas mthes  cnt o   r cehe clh aee oah
Length: 40
Epoch: 154
Ave Loss: 3.241887607436249
Generated sentence:
t   sh e sro h i  a oe hnhe  hie t  aeoh
Length: 40
Epoch: 155
Ave Loss: 3.241771428481392
Generated sentence:
ta  e hn ha  s   eo ersre aa   o  nt me 
Length: 40
Epoch: 156
Ave Loss: 3.2416582579773983
Generated sentence:
s ose fw a ah e he ae  ao neret  h   c a
Length: 40
Epoch: 157
Ave Loss: 3.2415483331910653
Generated sentence:
   saeea wt a e he  h  e e e e  he  hes 
Length: 40
Epoch: 158

Ave Loss: 3.2385601755501567
Generated sentence:
 te t s  hs an hei aa ee hr  hee ano  h 
Length: 40
Epoch: 224
Ave Loss: 3.238549056260482
Generated sentence:
ae  an  a e h e hin  ia  if ee l  h  aa 
Length: 40
Epoch: 225
Ave Loss: 3.238538366584962
Generated sentence:
 e hie  he shace hnrae  ae  ad e s ees l
Length: 40
Epoch: 226
Ave Loss: 3.238528103068255
Generated sentence:
 ae  ahse he  a h te refe  he  te ri s o
Length: 40
Epoch: 227
Ave Loss: 3.2385182230944793
Generated sentence:
a e  h  ed he eohi  taee hi en no er  t 
Length: 40
Epoch: 228
Ave Loss: 3.238508741636783
Generated sentence:
is lol ae i t  he a he  is feheset   ahe
Length: 40
Epoch: 229
Ave Loss: 3.2384995838294284
Generated sentence:
he nae ats  ase aa  em oee  no  io   h
 
Length: 40
Epoch: 230
Ave Loss: 3.2384908141721276
Generated sentence:
s eos h aace ar  hese  as  cr selha  ane
Length: 40
Epoch: 231
Ave Loss: 3.238482385441877
Generated sentence:
aahs e an  aa ee  he  ehse  in a aeee a 
Length: 40
Epoch: 2

Ave Loss: 3.238295951327264
Generated sentence:

ate cis eessa le hr fheln e s se ohel n
Length: 40
Epoch: 298
Ave Loss: 3.2382954664276418
Generated sentence:
t s h  aheee na he  hes ths ieheraha es 
Length: 40
Epoch: 299
Ave Loss: 3.23829502299212
Generated sentence:
 soeon he  th he  aaseena s ens ao d hes
Length: 40
Epoch: 300
Ave Loss: 3.2382946048957715
Generated sentence:
 o  e  eo iot ia aehh nl  ree ai  ae ese
Length: 40
Epoch: 301
Ave Loss: 3.2382941960136673
Generated sentence:
e en  e  ase oha aasete eaan  t  h a soe
Length: 40
Epoch: 302
Ave Loss: 3.2382938101671743
Generated sentence:
 tea ne hr ah l hi al eo eio  orde ian  
Length: 40
Epoch: 303
Ave Loss: 3.238293428927804
Generated sentence:
e ha tos oe  et hn   hnre he a aes cod h
Length: 40
Epoch: 304
Ave Loss: 3.2382930810900703
Generated sentence:
ee  hor s ais ane ha a se mae aie  he eh
Length: 40
Epoch: 305
Ave Loss: 3.238292737859459
Generated sentence:
reme  the ohh hahha hane en ee  a iooese
Length: 40
Epoch: 3

...

Ave Loss: 3.238285549596888
Generated sentence:
ie nha s ans oans toseh  aie  ae coan
 

Length: 40
Epoch: 668
Ave Loss: 3.238285549596888
Generated sentence:
c serfealhs niesre a see ae  ef resrtea 
Length: 40
Epoch: 669
Ave Loss: 3.238285549596888
Generated sentence:
eef ae  hs  otas  cr enf s  araselha oes
Length: 40
Epoch: 670
Ave Loss: 3.238285549596888
Generated sentence:
eo aa s ohe hhi alleeiha 
eiose  aae ohe
Length: 40
Epoch: 671
Ave Loss: 3.238285549596888
Generated sentence:
he  he  teroith he nha   d  eeeefit  or 
Length: 40
Epoch: 672
Ave Loss: 3.2382855484451074
Generated sentence:
eis the hnraed an tea e  he  anaees aa e
Length: 40
Epoch: 673
Ave Loss: 3.2382855484451074
Generated sentence:
 se ahe ra  re  are lo   he sao is aeaee
Length: 40
Epoch: 674
Ave Loss: 3.2382855484451074
Generated sentence:
ieh de anee s ais hnea aher he ee fae s 
Length: 40
Epoch: 675
Ave Loss: 3.2382855484451074
Generated sentence:
 efei aos hofshe aeal hima ne has i he
e
Length: 40
Epoch: 6

Ave Loss: 3.238285543837985
Generated sentence:
e eamsse o a he s m se mn ie tae sos i h
Length: 40
Epoch: 742
Ave Loss: 3.238285543837985
Generated sentence:
hna stfe  ha se arrh
aea inr  hn e toe  
Length: 40
Epoch: 743
Ave Loss: 3.238285543837985
Generated sentence:

oi o ae ae e  he cad hanteis oe s  ra h
Length: 40
Epoch: 744
Ave Loss: 3.238285543837985
Generated sentence:
rnaael sht aaeeors  a s aos  fos ae  ha 
Length: 40
Epoch: 745
Ave Loss: 3.238285543837985
Generated sentence:
 fo mote haseee nrr hein  ar  aoteehane 
Length: 40
Epoch: 746
Ave Loss: 3.238285543837985
Generated sentence:
e trois the ahr ha ealee ha  te sesolfie
Length: 40
Epoch: 747
Ave Loss: 3.238285543837985
Generated sentence:
fno  a ste hah oane chn a s io e fece
et
Length: 40
Epoch: 748
Ave Loss: 3.238285543837985
Generated sentence:
 aeae ans cttee  hs inha h ae sis e hate
Length: 40
Epoch: 749
Ave Loss: 3.238285543837985
Generated sentence:
 hee teeoristtihi aa sees er sate  han o
Length: 40
Epoch: 750
A

Ave Loss: 3.2382855426862043
Generated sentence:
n hlle  fee sin i heee mnra he  aad ai e
Length: 40
Epoch: 816
Ave Loss: 3.2382855426862043
Generated sentence:
 hasf hn aha maee ha nr no he  ann ch  f
Length: 40
Epoch: 817
Ave Loss: 3.2382855426862043
Generated sentence:
ad ae hes tnl el he  aoeeo se  ane  hnm 
Length: 40
Epoch: 818
Ave Loss: 3.2382855426862043
Generated sentence:
 hhe ha cemheeh sna sore  hani le chanre
Length: 40
Epoch: 819
Ave Loss: 3.2382855426862043
Generated sentence:
r aa ee ae aeolis these hass anoe
rad  i
Length: 40
Epoch: 820
Ave Loss: 3.238285541534424
Generated sentence:
t it  haneeit ii t eis td  aareehe ai ne
Length: 40
Epoch: 821
Ave Loss: 3.238285541534424
Generated sentence:
ns ca er nf ee hac oin  ha sehe ed nae e
Length: 40
Epoch: 822
Ave Loss: 3.238285541534424
Generated sentence:
sld
s a sas c sae har hls sieaoayor se e
Length: 40
Epoch: 823
Ave Loss: 3.238285541534424
Generated sentence:
 aes aed ahe  ae eaafsae lia   a serhs a
Length: 40
Epoch: 

Ave Loss: 3.238285541534424
Generated sentence:
e wa  hac chaoe anr iaor   feae oa i ane
Length: 40
Epoch: 890
Ave Loss: 3.238285541534424
Generated sentence:
o so 
  ha etha se siee shi  ionre aee  
Length: 40
Epoch: 891
Ave Loss: 3.238285541534424
Generated sentence:
nr  o te holefolne stn a her  mlse hads

Length: 40
Epoch: 892
Ave Loss: 3.238285541534424
Generated sentence:
ah  ta  hin e ardertis ane sooa   hnsamd
Length: 40
Epoch: 893
Ave Loss: 3.238285541534424
Generated sentence:
e se he ir orelye samae ohe  enne  hn cs
Length: 40
Epoch: 894
Ave Loss: 3.238285541534424
Generated sentence:
na in a mehe ahesrnehafer nan h h he she
Length: 40
Epoch: 895
Ave Loss: 3.238285541534424
Generated sentence:
lhm te othee oha  aerels d hio eh tol oe
Length: 40
Epoch: 896
Ave Loss: 3.238285541534424
Generated sentence:
s tore  hhsselo eis tee sais tae hrn  ha
Length: 40
Epoch: 897
Ave Loss: 3.238285541534424
Generated sentence:
ne oh me en  ea se taeia s ei n
 e  is l
Length: 40
Epoch: 898
A

Ave Loss: 3.238285541534424
Generated sentence:
anserfde aa ea ae or feaes  t  iaes f
 h
Length: 40
Epoch: 964
Ave Loss: 3.238285541534424
Generated sentence:
t if ne ls hi rorls he noa h hi gedn,ah 
Length: 40
Epoch: 965
Ave Loss: 3.238285541534424
Generated sentence:
oe h  en nnr mane  h e hll mesetadsats h
Length: 40
Epoch: 966
Ave Loss: 3.238285541534424
Generated sentence:
a taee nn  io o  ho ia fee aan ans ai e 
Length: 40
Epoch: 967
Ave Loss: 3.238285541534424
Generated sentence:
 e  as le lren hea ie  inh  hnr shsee he
Length: 40
Epoch: 968
Ave Loss: 3.238285541534424
Generated sentence:
h  oresfnef ae  hsp aesene  aes  ha san 
Length: 40
Epoch: 969
Ave Loss: 3.238285541534424
Generated sentence:
crhas anr se s t aeaeraae sanee ans is r
Length: 40
Epoch: 970
Ave Loss: 3.238285541534424
Generated sentence:
intc stse
el a s cae ha aeieishe  a
es s
Length: 40
Epoch: 971
Ave Loss: 3.238285541534424
Generated sentence:
eioht n  iae s a elinha a sloa
ee ahi  h
Length: 40
Epoch: 972
A